In [1]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
# Read Data Function
def read_data(filename):
    
    # Retrieve Data
    data = pd.read_csv(filename)
    
    # Drop Columns
    clean = data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Date'])
    
    # Drop Last Row
    clean = clean[:-1].copy()
    
    # Drop missing data
    clean = clean.dropna()
    
    # Rename Columns
    cols = ['Year', 'Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']
    clean.columns = cols
    
    return clean

In [3]:
# Run Model (Machine Learning)
def run_model(clean):
    
    # Create Lag Column
    clean['nextPFC'] = clean.shift(1)['PFC']
    
    # Set Training and Testing Split
    training = clean.loc[(clean['Year']<2010) & (clean['Year']>1990)]
    testing = clean.loc[clean['Year']>=2010]

    # Set Variables
    X_train = training[['Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']]
    y_train = training['nextPFC']
    X_test = testing[['Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']]
    y_test = testing['nextPFC']
    X = clean[['Population', 'CH4', 'CO2', 'GHG', 'HFC', 'N2O', 'PFC', 'SF6']]
    y = clean['nextPFC']

    # Linear Regression Model
    from sklearn.linear_model import LinearRegression
    
    model = LinearRegression()

    # Train Modele
    model.fit(X_train, y_train)

    # Score for 1991-2009
    print(f"Train Score: {model.score(X_train, y_train)}")

    # Score for 2010+
    print(f"Test Score: {model.score(X_test, y_test)}")

    # Predictions vs Actual
    predictions = model.predict(X)
    print(f"Actual: {y[1]}")
    print(f"Predicted: {predictions[1]}")
    print(f"Error: {predictions[1]-y[1]}")

    return pd.DataFrame({"Year": clean["Year"],
                         "Predicted": predictions,
                         "Actual": y,
                         "Error": predictions - y,
                         "Country": country})

In [4]:
final = pd.DataFrame(columns = ["Year", "Predicted", "Actual", "Error", "Country"])

for filename in os.listdir("Cleaning/Combined"):
    if ".csv" in filename:
        
        print()
        
        country = filename.replace('.csv', '')
        
        print(country)
        
        clean = read_data("Cleaning/Combined/" + filename)
        
        try:
            output = run_model(clean)
            final = final.append(output)
        
        except:
            print("Error")


Australia
Train Score: 0.9568470507353772
Test Score: -1186.9046628731135
Actual: 4607.01291909928
Predicted: 4972.305496046538
Error: 365.29257694725766

Canada
Train Score: 0.9911891191669779
Test Score: -948.4237900877395
Actual: 7557.901859348679
Predicted: 7633.165328248229
Error: 75.26346889955039

Japan
Train Score: 0.9260661540336409
Test Score: -60.69739635063029
Actual: 2906.85506987091
Predicted: 2826.9459339522073
Error: -79.90913591870276

New Zealand
Train Score: 0.8855533995095536
Test Score: -1645.1334924078897
Actual: 2662.8503218743303
Predicted: 2722.1414723120615
Error: 59.291150437731176

Russia
Train Score: 0.9729403156518871
Test Score: -230707.97076567818
Actual: 2808.42730048
Predicted: 2645.3658580113224
Error: -163.0614424686778

Spain
Train Score: 0.8402343878730856
Test Score: -52.637045111749735
Actual: 116.515249727925
Predicted: 116.43121458408609
Error: -0.08403514383890354

Switzerland
Train Score: 0.6900634838725415
Test Score: -6.178063347220041
Act

In [5]:
# Export 
final.to_csv('Resources/ML_PFC.csv')